2.2.1 创建tensor

In [1]:
import torch

In [2]:
x = torch.empty(5,3)
print(x)

tensor([[ 0.0000e+00,  4.6566e-10,  0.0000e+00],
        [ 4.6566e-10,  1.2860e-39,  4.5752e-41],
        [ 8.2461e-40,  4.5752e-41, -2.9860e+29],
        [ 4.5751e-41,  1.0586e-38,  1.4013e-45],
        [ 2.6652e-40,  4.5752e-41,  5.9298e-40]])


In [3]:
x = torch.rand(5,3)
print(x)

tensor([[0.9453, 0.0766, 0.3149],
        [0.7310, 0.1229, 0.8418],
        [0.4220, 0.6527, 0.3357],
        [0.4787, 0.9356, 0.3802],
        [0.2388, 0.7217, 0.1462]])


In [5]:
x = torch.zeros(5,3,dtype = torch.long)
print(x)

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])


In [6]:
x = torch.tensor([5.5,3])
print(x)

tensor([5.5000, 3.0000])


In [9]:
x = x.new_ones(5,3,dtype = torch.float64)
print(x)

x = torch.randn_like(x,dtype = torch.float)
print(x)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)
tensor([[ 0.4332,  1.2569, -0.7298],
        [ 1.1353,  0.4293,  1.5242],
        [ 0.9590, -1.9182, -1.1387],
        [ 0.0890, -0.1975, -0.6408],
        [-0.0059,  1.2850, -1.2481]])


In [10]:
print(x.size())

torch.Size([5, 3])


In [11]:
print(x.shape)

torch.Size([5, 3])


2.2.2 基本操作

In [12]:
y = torch.rand(5,3)
print(x + y)

tensor([[ 0.8018,  1.3688, -0.6557],
        [ 2.0463,  0.6960,  2.3315],
        [ 1.5811, -1.5944, -0.7284],
        [ 0.6968, -0.1854, -0.5198],
        [ 0.7661,  1.4852, -0.9330]])


In [13]:
result = torch.empty(5,3)
torch.add(x,y,out = result)
print(result)

tensor([[ 0.8018,  1.3688, -0.6557],
        [ 2.0463,  0.6960,  2.3315],
        [ 1.5811, -1.5944, -0.7284],
        [ 0.6968, -0.1854, -0.5198],
        [ 0.7661,  1.4852, -0.9330]])


索引 - 索引的结果与原数据共享内存，修改一个，另一个会跟着修改

In [14]:
y = x[0,:]
y += 1
print(y)

print(x[0,:])

tensor([1.4332, 2.2569, 0.2702])
tensor([1.4332, 2.2569, 0.2702])


改变形状 - 用view来改变tensor的形状

In [15]:
y = x.view(15)
z = x.view(-1,5)
print(x.shape,y.shape,z.shape)

torch.Size([5, 3]) torch.Size([15]) torch.Size([3, 5])


view返回的tensor 也是与原来的元组共享tensor的。改变其中的一个，另外一个也会跟着改变。

In [16]:
x += 1
print(x)
print(y)

tensor([[ 2.4332,  3.2569,  1.2702],
        [ 2.1353,  1.4293,  2.5242],
        [ 1.9590, -0.9182, -0.1387],
        [ 1.0890,  0.8025,  0.3592],
        [ 0.9941,  2.2850, -0.2481]])
tensor([ 2.4332,  3.2569,  1.2702,  2.1353,  1.4293,  2.5242,  1.9590, -0.9182,
        -0.1387,  1.0890,  0.8025,  0.3592,  0.9941,  2.2850, -0.2481])


如果想返回一个真正的副本，不共享data内存。推荐首先使用clone创建一个副本，然后再使用view。

In [17]:
x_cp = x.clone().view(15)
x -= 1

print(x)
print(x_cp)

tensor([[ 1.4332,  2.2569,  0.2702],
        [ 1.1353,  0.4293,  1.5242],
        [ 0.9590, -1.9182, -1.1387],
        [ 0.0890, -0.1975, -0.6408],
        [-0.0059,  1.2850, -1.2481]])
tensor([ 2.4332,  3.2569,  1.2702,  2.1353,  1.4293,  2.5242,  1.9590, -0.9182,
        -0.1387,  1.0890,  0.8025,  0.3592,  0.9941,  2.2850, -0.2481])


使用colne的好处，梯度传回副本时也会传到tensor

使用item可以将一个标量tensr转换成一个python number

In [18]:
x = torch.randn(1)

print(x)
print(x.item())

tensor([-2.1540])
-2.154015302658081


2.2.3 广播机制 - 适当复制元素到shape相同时进行计算

In [19]:
x = torch.arange(1,3).view(1,2)
print(x)

y = torch.arange(1,4).view(3,1)
print(y)

print(x + y)

tensor([[1, 2]])
tensor([[1],
        [2],
        [3]])
tensor([[2, 3],
        [3, 4],
        [4, 5]])


2.2.4 运算的内存开销

索引操作不会开辟内存，而像y = x+y等操作是会开辟新内存的，然后将y指向新内存。使用id函数，来验证内存地址是否相同

In [20]:
x = torch.tensor([1,2])
y = torch.tensor([3,4])

id_before = id(y)
y = y + x

print(id(y) == id_before) #false

False


如果想写回到原来的y的内存，可以使用索引进行替换 

In [21]:
x = torch.tensor([1,2])
y = torch.tensor([3,4])

id_before = id(y)
y[:] = y + x

print(id(y) == id_before) 

True


注意：
虽然view返回的tensor和原tensor共享data，但是二者的id（内存地址）并不一致，应该是用了某些映射机制。

2.2.5 tensor和numpy之间的相互转换

很容易使用numpy和from_numpy将tensor和numpy数组相互转换。但是需要注意：两个函数差生的tensor和numpy数组也会共享内存，其中一个改变时另一个也会改变。

使用torch.tensor()进行数据拷贝，并且不再共享内存。

In [22]:
a = torch.ones(5)
b = a.numpy()
print(a,b)

tensor([1., 1., 1., 1., 1.]) [1. 1. 1. 1. 1.]


In [23]:
a += 1
print(a,b)

tensor([2., 2., 2., 2., 2.]) [2. 2. 2. 2. 2.]


In [24]:
b += 1
print(a,b)

tensor([3., 3., 3., 3., 3.]) [3. 3. 3. 3. 3.]


In [26]:
c = torch.tensor(b)
a += 1
print(a,c)

tensor([5., 5., 5., 5., 5.]) tensor([4., 4., 4., 4., 4.])


2.2.6 Tensor on GPU

用to方法可以将tensor在cpu和gpu之间相互移动。

In [27]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    y = torch.ones_like(x,device = device)
    x = x.to(device)
    z = x + y
    print(z)
    print(z.to("cpu",torch.double))